# Quantum Optimization and Utilities Demo

This notebook demonstrates the quantum optimization and utility functions in quactuary:
- Circuit validation and analysis
- Circuit optimization techniques
- Parameter optimization for variational algorithms
- Performance estimation and resource planning

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple
import time

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import Parameter, ParameterVector
from qiskit.quantum_info import Statevector, random_unitary
from qiskit.visualization import circuit_drawer, plot_histogram

# Import quactuary quantum utilities
from quactuary.quantum.utils.validation import (
    validate_circuit_size,
    validate_probability_distribution,
    validate_quantum_state,
    validate_circuit_compatibility,
    validate_parameter_values,
    check_circuit_depth,
    estimate_circuit_runtime
)
from quactuary.quantum.utils.optimization import (
    optimize_circuit_depth,
    reduce_circuit_width,
    optimize_parameterized_circuit,
    estimate_optimization_speedup,
    find_optimal_decomposition,
    optimize_measurement_order,
    ParameterOptimizer
)
from quactuary.quantum.quantum_types import (
    CircuitMetrics,
    CircuitConstructionError,
    StatePreparationError,
    MAX_QUBITS_SIMULATOR
)
from quactuary.quantum.circuits.builders import CircuitBuilder, ParameterizedCircuitBuilder

## 1. Circuit Validation

Validate quantum circuits to ensure they meet requirements and constraints.

In [ ]:
# Example 1: Circuit size validation
print("Circuit Size Validation Examples:\n")

# Valid circuit sizes
for n_qubits in [5, 10, 20, 30]:
    try:
        validate_circuit_size(n_qubits, "simulator")
        print(f"✓ {n_qubits} qubits: Valid for simulator")
    except CircuitConstructionError as e:
        print(f"✗ {n_qubits} qubits: {e}")

# Test hardware limits
print("\nHardware validation:")
for n_qubits in [50, 100, 200]:
    try:
        validate_circuit_size(n_qubits, "hardware")
        print(f"✓ {n_qubits} qubits: Valid for hardware")
    except CircuitConstructionError as e:
        print(f"✗ {n_qubits} qubits: {e}")

In [ ]:
# Example 2: Probability distribution validation
print("Probability Distribution Validation:\n")

# Various test cases
test_distributions = [
    ([0.2, 0.3, 0.5], "Valid distribution"),
    ([0.1, 0.2, 0.3], "Unnormalized (sum < 1)"),
    ([0.5, 0.6, 0.7], "Unnormalized (sum > 1)"),
    ([0.25, -0.1, 0.85], "Contains negative value"),
    ([0, 0, 0], "All zeros")
]

for probs, description in test_distributions:
    print(f"{description}: {probs}")
    try:
        validated = validate_probability_distribution(probs)
        print(f"  ✓ Validated: {validated}")
        print(f"  Sum: {np.sum(validated):.10f}\n")
    except ValueError as e:
        print(f"  ✗ Error: {e}\n")

In [ ]:
# Example 3: Quantum state validation
print("Quantum State Validation:\n")

# Valid state
valid_state = np.array([1, 0, 0, 0]) / np.sqrt(1)
try:
    validate_quantum_state(valid_state, num_qubits=2)
    print("✓ Valid 2-qubit state |00>")
except StatePreparationError as e:
    print(f"✗ Error: {e}")

# Bell state
bell_state = np.array([1, 0, 0, 1]) / np.sqrt(2)
try:
    validate_quantum_state(bell_state, num_qubits=2)
    print("✓ Valid Bell state (|00> + |11>)/√2")
except StatePreparationError as e:
    print(f"✗ Error: {e}")

# Invalid: not normalized
unnormalized = np.array([1, 1, 1, 1])
try:
    validate_quantum_state(unnormalized)
    print("✗ Should have failed")
except StatePreparationError as e:
    print(f"✓ Correctly caught unnormalized state: {e}")

# Invalid: wrong dimension
wrong_dim = np.array([1, 0, 0])  # 3 is not a power of 2
try:
    validate_quantum_state(wrong_dim)
    print("✗ Should have failed")
except StatePreparationError as e:
    print(f"✓ Correctly caught wrong dimension: {e}")

In [ ]:
# Example 4: Parameter validation for variational circuits
print("Parameter Validation for Variational Circuits:\n")

# Create a parameterized circuit
params = ParameterVector('θ', 6)
qc = QuantumCircuit(3)
for i in range(3):
    qc.ry(params[i], i)
qc.cx(0, 1)
qc.cx(1, 2)
for i in range(3):
    qc.rz(params[i+3], i)

print(f"Circuit has {qc.num_parameters} parameters")

# Test various parameter values
test_params = [
    (np.random.rand(6), "Valid random parameters"),
    (np.ones(6) * np.pi, "Valid π parameters"),
    (np.ones(4), "Wrong number of parameters"),
    (np.array([1, 2, np.nan, 4, 5, 6]), "Contains NaN"),
    (np.array([1, 2, np.inf, 4, 5, 6]), "Contains infinity")
]

for param_values, description in test_params:
    print(f"\n{description}:")
    try:
        validate_parameter_values(qc, param_values)
        print(f"  ✓ Valid parameters: {param_values}")
    except ValueError as e:
        print(f"  ✗ Error: {e}")

## 2. Circuit Optimization

Optimize quantum circuits to reduce depth, gate count, and improve performance.

In [ ]:
# Create a complex circuit to optimize
def create_test_circuit(n_qubits=5):
    """Create a circuit with optimization opportunities."""
    qc = QuantumCircuit(n_qubits)
    
    # Add redundant gates
    for i in range(n_qubits):
        qc.h(i)
        qc.x(i)
        qc.x(i)  # Redundant: X·X = I
        qc.h(i)
        qc.h(i)  # Redundant: H·H = I
    
    # Add CNOT ladder
    for i in range(n_qubits-1):
        qc.cx(i, i+1)
        qc.cx(i, i+1)  # Redundant: CX·CX = I
    
    # Add rotations
    for i in range(n_qubits):
        qc.rz(np.pi/4, i)
        qc.rz(np.pi/4, i)  # Could be combined
    
    return qc

# Create and analyze original circuit
original = create_test_circuit(5)
print("Original Circuit:")
print(f"  Depth: {original.depth()}")
print(f"  Gate count: {len(original.data)}")
print(f"  Single-qubit gates: {sum(1 for inst in original.data if len(inst.qubits) == 1)}")
print(f"  Two-qubit gates: {sum(1 for inst in original.data if len(inst.qubits) == 2)}")

# Display circuit
original.draw(output='mpl', style='iqp')

In [ ]:
# Apply different optimization levels
optimization_results = []

for level in range(4):
    optimized = optimize_circuit_depth(original, optimization_level=level)
    
    metrics = CircuitMetrics.from_circuit(optimized)
    optimization_results.append({
        'level': level,
        'depth': metrics.depth,
        'gates': metrics.gate_count,
        'single_qubit': metrics.single_qubit_gates,
        'two_qubit': metrics.two_qubit_gates
    })
    
    if level == 3:  # Show the most optimized version
        print(f"\nOptimized Circuit (Level {level}):")
        print(f"  Depth: {metrics.depth}")
        print(f"  Gate count: {metrics.gate_count}")
        print(f"  Single-qubit gates: {metrics.single_qubit_gates}")
        print(f"  Two-qubit gates: {metrics.two_qubit_gates}")
        optimized.draw(output='mpl', style='iqp')

In [ ]:
# Visualize optimization results
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 10))

levels = [r['level'] for r in optimization_results]

# Circuit depth
ax1.bar(levels, [r['depth'] for r in optimization_results], color='skyblue')
ax1.axhline(original.depth(), color='red', linestyle='--', label='Original')
ax1.set_xlabel('Optimization Level')
ax1.set_ylabel('Circuit Depth')
ax1.set_title('Circuit Depth vs Optimization Level')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Total gate count
ax2.bar(levels, [r['gates'] for r in optimization_results], color='lightcoral')
ax2.axhline(len(original.data), color='red', linestyle='--', label='Original')
ax2.set_xlabel('Optimization Level')
ax2.set_ylabel('Total Gates')
ax2.set_title('Gate Count vs Optimization Level')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Gate breakdown
x = np.arange(len(levels))
width = 0.35

ax3.bar(x - width/2, [r['single_qubit'] for r in optimization_results], 
        width, label='Single-qubit', color='green')
ax3.bar(x + width/2, [r['two_qubit'] for r in optimization_results], 
        width, label='Two-qubit', color='orange')
ax3.set_xlabel('Optimization Level')
ax3.set_ylabel('Gate Count')
ax3.set_title('Gate Type Breakdown')
ax3.set_xticks(x)
ax3.set_xticklabels(levels)
ax3.legend()
ax3.grid(True, alpha=0.3)

# Optimization speedup
speedups = []
for r in optimization_results:
    speedup = estimate_optimization_speedup(
        original, 
        optimize_circuit_depth(original, r['level'])
    )
    speedups.append(speedup)

metrics_names = ['Depth\nReduction', 'Gate\nReduction', 'CNOT\nReduction']
level_3_speedup = speedups[3]

bars = ax4.bar(metrics_names, 
               [level_3_speedup['depth_reduction'], 
                level_3_speedup['gate_reduction'],
                level_3_speedup['cnot_reduction']], 
               color=['blue', 'red', 'green'])

# Add percentage labels
for bar in bars:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
             f'{height*100:.1f}%', ha='center', va='bottom')

ax4.set_ylabel('Reduction (%)')
ax4.set_title('Optimization Level 3 Improvements')
ax4.set_ylim([0, 1])
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Parameter Optimization for Variational Circuits

Optimize parameters in variational quantum algorithms.

In [ ]:
# Create a simple variational circuit for optimization
def create_variational_circuit(n_qubits=3):
    """Create a parameterized circuit for VQE-style optimization."""
    params = ParameterVector('θ', 2 * n_qubits)
    qc = QuantumCircuit(n_qubits)
    
    # First layer of rotations
    for i in range(n_qubits):
        qc.ry(params[i], i)
    
    # Entangling layer
    for i in range(n_qubits-1):
        qc.cx(i, i+1)
    
    # Second layer of rotations
    for i in range(n_qubits):
        qc.rz(params[i + n_qubits], i)
    
    return qc, params

# Define a simple cost function (finding ground state of Z⊗Z⊗Z)
def cost_function(params, circuit):
    """Cost function for parameter optimization."""
    # Bind parameters
    bound_circuit = circuit.assign_parameters(params)
    
    # Get statevector
    state = Statevector.from_instruction(bound_circuit)
    
    # Calculate expectation value of Z⊗Z⊗Z
    probs = state.probabilities()
    
    # Z⊗Z⊗Z eigenvalues: +1 for even parity, -1 for odd parity
    expectation = 0
    for i, prob in enumerate(probs):
        parity = bin(i).count('1') % 2
        eigenvalue = 1 if parity == 0 else -1
        expectation += eigenvalue * prob
    
    return -expectation  # Minimize negative expectation

# Create circuit and optimizer
var_circuit, var_params = create_variational_circuit(3)
print("Variational Circuit:")
var_circuit.draw(output='mpl', style='iqp')

In [ ]:
# Compare different optimization methods
optimization_methods = ['COBYLA', 'BFGS', 'Nelder-Mead']
results = {}

for method in optimization_methods:
    print(f"\nOptimizing with {method}...")
    
    # Create optimizer
    optimizer = ParameterOptimizer(method=method, max_iter=100)
    
    # Initial parameters
    initial_params = np.random.uniform(0, 2*np.pi, len(var_params))
    
    # Define cost function wrapper
    def wrapped_cost(params):
        return cost_function(params, var_circuit)
    
    # Optimize
    start_time = time.time()
    result = optimizer.optimize(wrapped_cost, initial_params)
    optimization_time = time.time() - start_time
    
    results[method] = {
        'result': result,
        'time': optimization_time,
        'history': optimizer.history
    }
    
    print(f"  Final cost: {result['optimal_value']:.6f}")
    print(f"  Iterations: {result['iterations']}")
    print(f"  Time: {optimization_time:.3f}s")
    print(f"  Converged: {result['converged']}")

In [ ]:
# Visualize optimization convergence
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Convergence curves
for method, data in results.items():
    history = data['history']
    iterations = [h['iteration'] for h in history]
    costs = [h['cost'] for h in history]
    
    ax1.plot(iterations, costs, label=method, linewidth=2)

ax1.set_xlabel('Iteration')
ax1.set_ylabel('Cost Function Value')
ax1.set_title('Optimization Convergence')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')

# Performance comparison
methods = list(results.keys())
times = [results[m]['time'] for m in methods]
final_costs = [results[m]['result']['optimal_value'] for m in methods]
iterations = [results[m]['result']['iterations'] for m in methods]

x = np.arange(len(methods))
width = 0.35

ax2.bar(x - width/2, times, width, label='Time (s)', color='skyblue')
ax2.bar(x + width/2, np.array(iterations)/20, width, 
        label='Iterations/20', color='lightcoral')

ax2.set_xlabel('Optimization Method')
ax2.set_ylabel('Value')
ax2.set_title('Optimization Performance')
ax2.set_xticks(x)
ax2.set_xticklabels(methods)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Circuit Width Reduction

Optimize circuits by reducing the number of qubits when possible.

In [ ]:
# Create a circuit with unused qubits
def create_sparse_circuit():
    """Create a circuit where not all qubits are used."""
    qc = QuantumCircuit(8)  # 8 qubits allocated
    
    # Only use qubits 0, 2, 5
    qc.h(0)
    qc.x(2)
    qc.y(5)
    qc.cx(0, 2)
    qc.cz(2, 5)
    
    return qc

# Create and analyze sparse circuit
sparse_circuit = create_sparse_circuit()
print("Original Sparse Circuit:")
print(f"  Total qubits: {sparse_circuit.num_qubits}")
print(f"  Used qubits: {sorted(set(q.index for inst in sparse_circuit.data for q in inst.qubits))}")

# Reduce width
reduced_circuit = reduce_circuit_width(sparse_circuit)
print("\nReduced Circuit:")
print(f"  Total qubits: {reduced_circuit.num_qubits}")
print(f"  Circuit is functionally equivalent")

# Visualize both
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
sparse_circuit.draw(output='mpl', ax=ax1, style='iqp')
ax1.set_title('Original (8 qubits)')

reduced_circuit.draw(output='mpl', ax=ax2, style='iqp')
ax2.set_title('Reduced (3 qubits)')

plt.tight_layout()
plt.show()

## 5. Measurement Order Optimization

Optimize the order of measurements to reduce circuit depth.

In [ ]:
# Create a circuit with suboptimal measurement order
def create_measurement_circuit():
    """Create a circuit where measurement order can be optimized."""
    qc = QuantumCircuit(5, 5)
    
    # Operations on different qubits
    qc.h(0)
    qc.x(1)
    
    # Qubit 0 is done early
    qc.barrier()
    
    # More operations on other qubits
    qc.cx(1, 2)
    qc.cy(2, 3)
    qc.cz(3, 4)
    
    # Measurements in suboptimal order
    qc.measure(4, 4)  # Measure last qubit first
    qc.measure(3, 3)
    qc.measure(2, 2)
    qc.measure(1, 1)
    qc.measure(0, 0)  # Measure first qubit last
    
    return qc

# Create and optimize
original_meas = create_measurement_circuit()
optimized_meas = optimize_measurement_order(original_meas)

print("Measurement Order Optimization:")
print(f"Original depth: {original_meas.depth()}")
print(f"Optimized depth: {optimized_meas.depth()}")
print(f"Depth reduction: {original_meas.depth() - optimized_meas.depth()}")

# Visualize
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
original_meas.draw(output='mpl', ax=ax1, style='iqp')
ax1.set_title('Original Measurement Order')

optimized_meas.draw(output='mpl', ax=ax2, style='iqp')
ax2.set_title('Optimized Measurement Order')

plt.tight_layout()
plt.show()

## 6. Runtime Estimation

Estimate circuit execution time for different backends.

In [ ]:
# Create circuits of varying complexity
test_circuits = []

# Simple circuit
simple = QuantumCircuit(3)
simple.h(range(3))
simple.cx(0, 1)
simple.cx(1, 2)
test_circuits.append(("Simple (3q)", simple))

# Medium circuit
medium = QuantumCircuit(5)
for _ in range(3):
    medium.h(range(5))
    for i in range(4):
        medium.cx(i, i+1)
    medium.barrier()
test_circuits.append(("Medium (5q)", medium))

# Complex circuit
complex_circuit = QuantumCircuit(8)
for _ in range(5):
    complex_circuit.h(range(8))
    for i in range(7):
        complex_circuit.cx(i, i+1)
    for i in range(8):
        complex_circuit.rz(np.pi/4, i)
test_circuits.append(("Complex (8q)", complex_circuit))

# Estimate runtime for each circuit
print("Runtime Estimation Analysis")
print("=" * 70)
print(f"{'Circuit':<15} {'Qubits':<8} {'Depth':<8} {'Gates':<8} "
      f"{'Sim Time (ms)':<15} {'HW Time (μs)':<15}")
print("-" * 70)

runtime_data = []
for name, circuit in test_circuits:
    sim_time = estimate_circuit_runtime(circuit, "simulator") * 1000  # Convert to ms
    hw_time = estimate_circuit_runtime(circuit, "hardware") * 1e6   # Convert to μs
    
    runtime_data.append({
        'name': name,
        'qubits': circuit.num_qubits,
        'depth': circuit.depth(),
        'gates': len(circuit.data),
        'sim_time': sim_time,
        'hw_time': hw_time
    })
    
    print(f"{name:<15} {circuit.num_qubits:<8} {circuit.depth():<8} "
          f"{len(circuit.data):<8} {sim_time:<15.3f} {hw_time:<15.1f}")

In [ ]:
# Visualize scaling behavior
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 10))

# Test scaling with qubit number
qubit_range = range(2, 16)
sim_times = []
hw_times = []

for n in qubit_range:
    qc = QuantumCircuit(n)
    qc.h(range(n))
    for i in range(n-1):
        qc.cx(i, i+1)
    
    sim_times.append(estimate_circuit_runtime(qc, "simulator") * 1000)
    hw_times.append(estimate_circuit_runtime(qc, "hardware") * 1e6)

# Simulator scaling (exponential)
ax1.semilogy(qubit_range, sim_times, 'bo-', markersize=8)
ax1.set_xlabel('Number of Qubits')
ax1.set_ylabel('Estimated Time (ms)')
ax1.set_title('Simulator Runtime Scaling')
ax1.grid(True, alpha=0.3)

# Hardware scaling (linear with depth)
ax2.plot(qubit_range, hw_times, 'ro-', markersize=8)
ax2.set_xlabel('Number of Qubits')
ax2.set_ylabel('Estimated Time (μs)')
ax2.set_title('Hardware Runtime Scaling')
ax2.grid(True, alpha=0.3)

# Circuit metrics for runtime data
names = [d['name'] for d in runtime_data]
x = np.arange(len(names))

ax3.bar(x, [d['sim_time'] for d in runtime_data], color='blue', alpha=0.7)
ax3.set_xlabel('Circuit Type')
ax3.set_ylabel('Simulator Time (ms)')
ax3.set_title('Estimated Simulator Runtime')
ax3.set_xticks(x)
ax3.set_xticklabels(names)
ax3.grid(True, alpha=0.3)

ax4.bar(x, [d['hw_time'] for d in runtime_data], color='red', alpha=0.7)
ax4.set_xlabel('Circuit Type')
ax4.set_ylabel('Hardware Time (μs)')
ax4.set_title('Estimated Hardware Runtime')
ax4.set_xticks(x)
ax4.set_xticklabels(names)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Advanced: Optimal Decomposition

Find optimal decomposition of unitaries into basis gates.

In [ ]:
# Create a random 2-qubit unitary
unitary = random_unitary(4).data  # 4x4 matrix for 2 qubits

# Define different basis gate sets
basis_sets = [
    (['cx', 'rz', 'sx', 'x'], "IBM basis"),
    (['cx', 'u1', 'u2', 'u3'], "Universal basis"),
    (['cx', 'ry', 'rz'], "Minimal basis")
]

print("Optimal Unitary Decomposition")
print("=" * 60)

decomposition_results = []
for basis_gates, name in basis_sets:
    try:
        # Find optimal decomposition
        decomposed = find_optimal_decomposition(unitary, basis_gates)
        
        metrics = CircuitMetrics.from_circuit(decomposed)
        decomposition_results.append({
            'name': name,
            'basis': basis_gates,
            'depth': metrics.depth,
            'gates': metrics.gate_count,
            'cnots': metrics.cnot_count
        })
        
        print(f"\n{name}:")
        print(f"  Basis gates: {basis_gates}")
        print(f"  Circuit depth: {metrics.depth}")
        print(f"  Total gates: {metrics.gate_count}")
        print(f"  CNOT gates: {metrics.cnot_count}")
        
    except Exception as e:
        print(f"\n{name}: Failed - {e}")

## 8. Parameter Merging for Variational Circuits

Optimize parameterized circuits by merging similar parameters.

In [ ]:
# Create a parameterized circuit with redundant parameters
n_params = 12
params = ParameterVector('θ', n_params)
qc = QuantumCircuit(4)

# Add parameterized gates
for i in range(3):
    for j in range(4):
        qc.ry(params[i*4 + j], j)
    if i < 2:
        qc.cx(0, 1)
        qc.cx(2, 3)

print(f"Original circuit: {qc.num_parameters} parameters")

# Create parameter values with some similar values
param_values = np.array([
    0.5, 0.5, 1.0, 1.5,  # First layer
    0.5, 0.51, 1.0, 2.0,  # Second layer (some similar to first)
    1.5, 1.49, 0.5, 1.0   # Third layer (some similar to others)
])

# Optimize by merging similar parameters
optimized_circuit, optimized_values = optimize_parameterized_circuit(
    qc, param_values, merge_threshold=0.02
)

print(f"\nOptimized circuit: {optimized_circuit.num_parameters} parameters")
print(f"Parameter reduction: {qc.num_parameters - optimized_circuit.num_parameters}")
print(f"\nOriginal values: {param_values}")
print(f"Optimized values: {optimized_values}")

# Verify functional equivalence
original_bound = qc.assign_parameters(param_values)
optimized_bound = optimized_circuit.assign_parameters(optimized_values)

original_state = Statevector.from_instruction(original_bound)
optimized_state = Statevector.from_instruction(optimized_bound)

fidelity = np.abs(np.dot(original_state.data.conj(), optimized_state.data))**2
print(f"\nFidelity between original and optimized: {fidelity:.10f}")

## Summary

This notebook demonstrated quantum optimization and utility functions:

1. **Circuit Validation**:
   - Size validation for different backends
   - Probability distribution normalization
   - Quantum state validation
   - Parameter validation for variational circuits

2. **Circuit Optimization**:
   - Depth reduction at different optimization levels
   - Gate cancellation and combination
   - Performance speedup estimation

3. **Parameter Optimization**:
   - Different classical optimization methods
   - Convergence analysis
   - Performance comparison

4. **Advanced Techniques**:
   - Circuit width reduction
   - Measurement order optimization
   - Runtime estimation for different backends
   - Optimal unitary decomposition
   - Parameter merging for variational circuits

Key takeaways:
- Validation ensures circuits meet backend constraints
- Optimization can significantly reduce circuit depth and gate count
- Runtime scales exponentially with qubits for simulators, linearly for hardware
- Parameter optimization choice affects convergence speed
- Advanced techniques can further improve circuit efficiency